In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# libraries import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

import missingno as msno
import warnings
warnings.filterwarnings('ignore')

from scipy import stats

plt.rcParams["font.family"] = "NanumBarunGothic"

In [ ]:
# 지도 그리기
!pip install folium
import folium

In [ ]:
# 한글 폰트 설치

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
plt.rc('font', family='NanumBarunGothic')

In [ ]:
# data import

# 감염병 빅데이터 거래소

by_year_patient = pd.read_csv('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/연령별 감염병 환자정보/CD122200020001/TB_LM_AGRDE_CNT_S.csv')
by_region_patient = pd.read_csv('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/지역별 감염병 환자정보/CD122200010001/TB_LM_RGN_CNT_S.csv')
by_year_patient_percentage = pd.read_csv('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/연령별 감염병 환자 발생 비율/CD122200060001/TB_LM_AGRDE_RATE_S.csv')
by_region_patient_percentage = pd.read_csv('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/지역별 감염병 환자 발생 비율/CD122200050001/TB_LM_RGN_RATE_S.csv')

# 행정안전부 주민등록 인구통계

population_data = pd.read_csv('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/주민등록 인구 현황/202201_202212_주민등록인구기타현황(고령 인구현황)_OldAge.csv' , encoding = 'cp949')

# 응급의료통계포털 MEDIS

by_region_facility = pd.read_excel('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/시도별 응급의료기관 및 응급의료 시설/응급의료기관 및 응급의료시설 현황 - 시도별, 연도별.xlsx')
by_day_facility_using = pd.read_excel('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/내원요일별 응급실 이용 (성별 + 연령별)/내원요일별 응급실 이용(성별, 연령별).xlsx')
by_day_facility_using_region = pd.read_excel('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/내원요일별 응급실 이용 (시도별)/내원요일별 응급실 이용(시도별).xlsx')

# KOSIS

alone_percentage = pd.read_excel('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/독거노인 가구 비율 (시도-시-군-구)/독거노인가구비율_시도_시_군_구__20231102231600.xlsx')

## 데이터 분석 프로젝트

- 1. 데이터 살펴보기
- 2. 데이터별 분석 + 전처리
- 3. 데이터 구성하기 (정리) + 파생변수 제작
- 4. 관계성 분석
- 5. 군집

### 1. 데이터 살펴보기

#### 1-1. 감염병 빅데이터 거래소

In [ ]:
# by_year_patient

by_year_patient.head(5)

In [ ]:
by_year_patient.info()

In [ ]:
"""
모델 만들 경우
정규화 과정 필요
"""

by_year_patient[['infcd_ptnt_cnt' , 'agrde_rate']].describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
# by_region_patient

by_region_patient.head(5)

In [ ]:
by_region_patient.info()

In [ ]:
by_region_patient[["infcd_ptnt_cnt" , "rgn_rate"]].describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
# by_year_patient_percentage

by_year_patient_percentage.head(5)

In [ ]:
by_year_patient_percentage.info()

In [ ]:
"""
count를 보면 NA값이 존재하기에 이후에 비율 계산
NA처리를 조심해서
"""

by_year_patient_percentage[["infcd_ptnt_cnt" , "infcd_ptnt_rate" , "whol_ptnt_cnt"]].describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
# by_region_patient_percentage

by_region_patient_percentage.head(5)

In [ ]:
by_region_patient_percentage.info()

In [ ]:
by_region_patient_percentage[["infcd_ptnt_cnt" , "infcd_ptnt_rate" , "whol_ptnt_cnt"]].describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
# strd_yr , stnmt 공통적으로 존재하기 때문에 연-월로 datetime 만들어서
# 시계열 분석하기

#### 1-2. 행정안전부 주민등록 인구통계

In [ ]:
population_data.head(5)

In [ ]:
population_data.info()

In [ ]:
"""
행정구역 컬럼에서 필요없는 부분 없애기 ,
타입 바꾸기
"""

def type_change(start , data):
    for col in data.columns[start:]:
        for idx in range(len(data[col])):
            data[col][idx] = "".join(data[col][idx].split(','))
        data[col] = data[col].astype('int')
    return data

In [ ]:
population_data = type_change(1 , population_data)

In [ ]:
population_data.iloc[: , 1:].describe().style.background_gradient(cmap = 'summer_r')

#### 1-3. 응급의료통계포털 MEDIS

In [ ]:
# by_region_facility

by_region_facility.head(5)

In [ ]:
by_region_facility.info()

In [ ]:
by_region_facility.iloc[: , 4:].describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
# by_day_facility_using

by_day_facility_using.head(5)

In [ ]:
by_day_facility_using.info()

In [ ]:
by_day_facility_using = type_change(2,  by_day_facility_using)

In [ ]:
by_day_facility_using.iloc[: , 2:].describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
# by_day_facility_using_region

by_day_facility_using_region.head(5)

In [ ]:
by_day_facility_using_region.info()

In [ ]:
by_day_facility_using_region = type_change(2,  by_day_facility_using_region)

In [ ]:
by_day_facility_using_region.iloc[: , 2:].describe().style.background_gradient(cmap = 'summer_r')

#### 1-4. KOSIS

In [ ]:
# alone_percentage

alone_percentage.head(5)

In [ ]:
alone_percentage.info()

In [ ]:
alone_percentage.describe().style.background_gradient(cmap = 'summer_r')

### 2. 데이터별 분석 + 전처리

#### 2-1. 감염병 빅데이터 거래소

In [ ]:
# by_year_patient

by_year_patient.head(3)

by_year_patient.drop(['sqno'] , axis = 1 , inplace = True)
by_year_patient['time'] = pd.to_datetime(by_year_patient['strd_yr'].astype('str') + by_year_patient['stnmt'].astype('str') , format = '%Y%m').dt.strftime("%Y-%m")

In [ ]:
# by_region_patient

by_region_patient.head(3)

by_region_patient.drop(['sqno'] , axis = 1 , inplace = True)
by_region_patient['time'] = pd.to_datetime(by_region_patient['strd_yr'].astype('str') + by_region_patient['stnmt'].astype('str') , format = '%Y%m').dt.strftime("%Y-%m")

In [ ]:
# by_year_patient_percentage

by_year_patient_percentage.head(3)

by_year_patient_percentage.drop(['sqno'] , axis = 1 , inplace = True)
by_year_patient_percentage['time'] = pd.to_datetime(by_year_patient_percentage['strd_yr'].astype('str') + by_year_patient_percentage['stnmt'].astype('str') , format = '%Y%m').dt.strftime("%Y-%m")

In [ ]:
# by_region_patient_percentage

by_region_patient_percentage.head(3)

by_region_patient_percentage.drop(['sqno'] , axis = 1 , inplace = True)
by_region_patient_percentage['time'] = pd.to_datetime(by_region_patient_percentage['strd_yr'].astype('str') + by_region_patient_percentage['stnmt'].astype('str') , format = '%Y%m').dt.strftime("%Y-%m")

##### by_year_patient

In [ ]:
# by_year_patient 단일 분석

by_year_patient.head(3)

In [ ]:
fig , ax = plt.subplots(2 , 2 , figsize = (8 , 8))

sns.countplot( # 월 비중
    data = by_year_patient ,
    x = 'stnmt' ,
    ax = ax[0][0]
)

sns.countplot( # 나이 비중
    data = by_year_patient ,
    x = 'agrde_cd' ,
    ax = ax[0][1]
)

sns.histplot(
    data = by_year_patient ,
    x = 'infcd_ptnt_cnt' ,
    ax = ax[1][0] ,
    kde = True
)

sns.histplot(
    data = by_year_patient ,
    x = 'agrde_rate' ,
    ax = ax[1][1] ,
    kde = True
)

In [ ]:
# 이상치

sns.boxplot(
    y = by_year_patient['infcd_ptnt_cnt']
)

In [ ]:
# 이상치 추출하는 함수

def outlier_extractor(data , col):

    q1 = np.percentile(data[col] , 25)
    q3 = np.percentile(data[col] , 75)

    IQR = q3 - q1

    upper_fence = q3 + 1.5 * IQR
    lower_fence = q1 - 1.5 * IQR

    return data[(data[col] < lower_fence) | (data[col] > upper_fence)]

In [ ]:
# 이상치 데이터

outlier = outlier_extractor(by_year_patient , 'infcd_ptnt_cnt')
outlier

In [ ]:
fig , ax = plt.subplots(1 , 2 , figsize = (8 , 5))

data = by_year_patient.groupby(['agrde_cd'])['infcd_ptnt_cnt'].mean() # 연령별

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[0]
)

data = by_year_patient.groupby(['stnmt'])['infcd_ptnt_cnt'].mean() # 날짜별

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[1]
)


##### by_region_patient

In [ ]:
# by_region_patient 단일분석

by_region_patient.head(5)

In [ ]:
fig , ax = plt.subplots(2 , 2 , figsize = (8 , 8))

sns.countplot(
    data = by_region_patient ,
    x = 'stnmt' ,
    ax = ax[0][0]
)

sns.countplot(
    data = by_region_patient ,
    x = 'rgn_cd' ,
    ax = ax[0][1]
)

sns.histplot(
    data = by_region_patient ,
    x = 'infcd_ptnt_cnt' ,
    ax = ax[1][0] ,
    kde = True
)

sns.histplot(
    data = by_region_patient ,
    x = 'rgn_rate' ,
    ax = ax[1][1] ,
    kde = True
)

In [ ]:
# 이상치

fig , ax = plt.subplots(1 , 2)

sns.boxplot(
    y = by_region_patient['infcd_ptnt_cnt'] ,
    ax = ax[0]
)

sns.boxplot(
    y = by_region_patient['rgn_rate'] ,
    ax = ax[1]
)

In [ ]:
outlier_infcd_ptnt_cnt = outlier_extractor(by_region_patient , 'infcd_ptnt_cnt')
outlier_rgn_rate = outlier_extractor(by_region_patient , 'rgn_rate')

In [ ]:
outlier_infcd_ptnt_cnt

In [ ]:
outlier_rgn_rate

In [ ]:
fig , ax = plt.subplots(1 , 2 , figsize = (8 , 5))

data = by_region_patient.groupby(['rgn_cd'])['infcd_ptnt_cnt'].mean()

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[0]
)

data = by_region_patient.groupby(['rgn_cd'])['rgn_rate'].mean()

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[1]
)

##### by_year_patient_percentage

In [ ]:
# by_year_patient_percentage  단일분석

by_year_patient_percentage.head(5)

In [ ]:
fig , ax = plt.subplots(2 , 2 , figsize = (8 , 8))

col = ['stnmt' , 'agrde_cd' , 'infcd_ptnt_rate' , 'whol_ptnt_cnt']

for idx in range(len(col)):
    row = idx // 2
    cool = idx % 2

    if row == 0:
        sns.countplot(
            data = by_year_patient_percentage ,
            x = col[idx] ,
            ax = ax[row][cool]
        )
    else:
        sns.histplot(
            data = by_year_patient_percentage ,
            x = col[idx] ,
            kde = True ,
            ax = ax[row][cool]
        )

In [ ]:
# 이상치

fig , ax = plt.subplots(1 , 2)

sns.boxplot(
    y = by_year_patient_percentage['infcd_ptnt_rate'] ,
    ax = ax[0]
)

sns.boxplot(
    y = by_year_patient_percentage['whol_ptnt_cnt'] ,
    ax = ax[1]
)

In [ ]:
fig , ax = plt.subplots(1 , 3 , figsize = (12 , 5))

data = by_year_patient_percentage.groupby(['agrde_cd'])['infcd_ptnt_cnt'].mean()

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[0]
)

data = by_year_patient_percentage.groupby(['agrde_cd'])['infcd_ptnt_rate'].mean()

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[1]
)

data = by_year_patient_percentage.groupby(['agrde_cd'])['whol_ptnt_cnt'].mean()

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[2]
)

##### by_region_patient_percentage

In [ ]:
# by_region_patient_percentage 단일분석

by_region_patient_percentage.head(5)

In [ ]:
by_region_patient_percentage.groupby(['rgn_cd'])['infcd_ptnt_cnt'].mean()

In [ ]:
fig , ax = plt.subplots(2 , 2 , figsize = (8 , 8))

col = ['stnmt' , 'rgn_cd' , 'infcd_ptnt_rate' , 'whol_ptnt_cnt']

for idx in range(len(col)):
    row = idx // 2
    cool = idx % 2

    if row == 0:
        sns.countplot(
            data = by_region_patient_percentage ,
            x = col[idx] ,
            ax = ax[row][cool]
        )
    else:
        sns.histplot(
            data = by_region_patient_percentage ,
            x = col[idx] ,
            kde = True ,
            ax = ax[row][cool]
        )

In [ ]:
# 이상치

fig , ax = plt.subplots(1 , 2)

sns.boxplot(
    y = by_region_patient_percentage['infcd_ptnt_rate'] ,
    ax = ax[0]
)

sns.boxplot(
    y = by_region_patient_percentage['whol_ptnt_cnt'] ,
    ax = ax[1]
)

In [ ]:
# 이상치 추출

outlier_rate = outlier_extractor(by_region_patient_percentage , 'infcd_ptnt_rate')
outlier_whol = outlier_extractor(by_region_patient_percentage , 'whol_ptnt_cnt')

In [ ]:
outlier_whol

In [ ]:
fig , ax = plt.subplots(1 , 3 , figsize = (12 , 5))

data = by_region_patient_percentage.groupby(['rgn_cd'])['infcd_ptnt_cnt'].mean()

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[0]
)

data = by_region_patient_percentage.groupby(['rgn_cd'])['infcd_ptnt_rate'].mean()

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[1]
)

data = by_region_patient_percentage.groupby(['rgn_cd'])['whol_ptnt_cnt'].mean()

sns.barplot(
    x = data.index ,
    y = data.values ,
    ax = ax[2]
)

#### 2-2. 행정안전부 주민등록 인구통계

In [ ]:
# population_data 단일분석

population_data.head(5)

In [ ]:
# 성별별로 분석을 진행하지는 않기 때문에 전체 데이터만 남겨 놓는다.

for month in range(1 , 13):
    if month < 10:
        population_data.drop([f"2022년0{month}월_남자" , f"2022년0{month}월_여자" , f"2022년0{month}월_65세이상남자" , f"2022년0{month}월_65세이상여자"] , axis = 1 , inplace = True)
    else:
        population_data.drop([f"2022년{month}월_남자" , f"2022년{month}월_여자" , f"2022년{month}월_65세이상남자" , f"2022년{month}월_65세이상여자"] , axis = 1 , inplace = True)

In [ ]:
# 추이 분석

region = population_data.iloc[: , 0]

under_65 = population_data.iloc[: , [i for i in range(1 , 25 , 2)]]
over_65 = population_data.iloc[: , [i for i in range(2 , 25 , 2)]]

under_65 = pd.concat([region , under_65] , axis = 1)
over_65 = pd.concat([region , over_65] , axis = 1)

In [ ]:
over_65.index = over_65['행정구역']
over_65.drop(['행정구역'] ,  axis = 1 , inplace = True)

In [ ]:
under_65.index = under_65['행정구역']
under_65.drop(['행정구역'] , axis = 1 , inplace = True)

In [ ]:
# 65세 이상

plt.figure(figsize = (12 , 12))

for i in range(1 , 18):
    plt.plot(
        over_65.columns ,
        over_65.iloc[i , :] ,
        label = over_65.index[i]
    )

    plt.text(
        over_65.columns[-1] ,
        over_65.iloc[i , :][-1] ,
        over_65.index[i]
    )
plt.xticks(rotation = 45)

In [ ]:
# 65세 이하

plt.figure(figsize = (12 , 12))

for i in range(1 , 18):
    plt.plot(
        under_65.columns ,
        under_65.iloc[i , :] ,
        label = under_65.index[i]
    )

    plt.text(
        under_65.columns[-1] ,
        under_65.iloc[i , :][-1] ,
        under_65.index[i]
    )

plt.xticks(rotation = 45)

In [ ]:
# 비율 보기

new_over = over_65.copy()
new_under = under_65.copy()

for idx in range(len(new_over.columns)):
    new_over.rename(columns = {new_over.columns[idx] : str(idx + 1)} , inplace = True)
    new_under.rename(columns = {new_under.columns[idx] : str(idx + 1)} , inplace = True)

In [ ]:
# 새로운 데이터프레임 (비율) 제작
new_percent = new_over / new_under

In [ ]:
# 시간

plt.figure(figsize = (12 , 12))

for i in range(1 , 18):
    plt.plot(
        new_percent.columns ,
        new_percent.iloc[i , :] ,
        label = new_percent.index[i]
    )

    plt.text(
        new_percent.columns[-1] ,
        new_percent.iloc[i , :][-1] ,
        new_percent.index[i]
    )

plt.xticks(rotation = 45)

#### 2-3. 응급의료통계포털 MEDIS

##### by_region_facility

In [ ]:
# by_region_facility

by_region_facility.head(5)

In [ ]:
by_region_facility.info()

In [ ]:
# NA처리

"""
응급의료기관외의 응급실 운영기관 drop
소아전문응급의료센터 drop
화상전문응급의료센터 drop
"""

for col in by_region_facility.columns[4 :]:
    print(f"{col} NA : {((by_region_facility[col].isna().sum()) / (len(by_region_facility))) * 100}%")

by_region_facility.drop(['응급의료기관외의 응급실 운영기관' , '소아전문응급의료센터' , '화상전문응급의료센터'] , axis = 1 , inplace = True)

by_region_facility['권역응급의료센터'].fillna(by_region_facility['권역응급의료센터'].mean() , inplace = True)
by_region_facility['응급의료시설'].fillna(by_region_facility['응급의료시설'].mean() , inplace = True)
by_region_facility['권역외상센터'].fillna(by_region_facility['권역외상센터'].mean() , inplace = True)

In [ ]:
# 필요없는 컬럼 제거

by_region_facility.drop(['No' , '계' , '소계'] , axis = 1 , inplace = True)

In [ ]:
by_region_facility.describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
# by_region_facility

fig , ax = plt.subplots(3 , 2 , figsize = (8 , 8))

for idx in range(len(by_region_facility.columns) - 1):
    row = idx // 2
    col = idx % 2

    sns.histplot(
        data = by_region_facility ,
        x = by_region_facility.columns[idx + 1] ,
        ax = ax[row][col] ,
        kde = True
    )

In [ ]:
by_region_facility

In [ ]:
# 지역별 시각화

fig , ax = plt.subplots(2 , 2 , figsize = (13 , 8))

for i in range(4):
    row = i // 2
    col = i % 2

    sns.barplot(
        data = by_region_facility ,
        x =  "분류" ,
        y = by_region_facility.columns[i + 1] ,
        ax = ax[row][col]
    )

##### by_day_facility_using

In [ ]:
# by_day_facility_using

by_day_facility_using.head(5)

In [ ]:
by_day_facility_using.info()

In [ ]:
# 필요없는 컬럼 제거

by_day_facility_using.drop(['No'] , axis = 1 , inplace = True)

In [ ]:
by_day_facility_using.describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
fig , ax = plt.subplots(2 , 4 , figsize = (13 , 10))

for i in range(8):
    row = i // 4
    col = i % 4

    sns.histplot(
        data = by_day_facility_using ,
        x = by_day_facility_using.columns[i + 1] ,
        ax = ax[row][col] ,
        kde = True
    )

In [ ]:
plt.figure(figsize = (8 , 8))

age_data = by_day_facility_using.iloc[3 : 13 , :]

column = age_data.columns[2 : ]

for idx in range(10):
    plt.plot(
        column ,
        age_data.iloc[idx][2 : ] ,
        label = age_data.iloc[idx][0]
    )

    plt.text(
        column[-1] ,
        age_data.iloc[idx][-1] ,
        age_data.iloc[idx][0]
    )
plt.xticks(rotation = 45)

##### by_day_facility_using_reigon

In [ ]:
# by_day_facility_using_region

by_day_facility_using_region.head(5)

In [ ]:
by_day_facility_using_region.info()

In [ ]:
by_day_facility_using_region.drop(['No'] , axis = 1 , inplace = True)

In [ ]:
by_day_facility_using_region.describe().style.background_gradient(cmap = 'summer_r')

In [ ]:
fig , ax = plt.subplots(2 , 4 , figsize = (13 , 10))

for i in range(8):
    row = i // 4
    col = i % 4

    sns.histplot(
        data = by_day_facility_using_region ,
        x = by_day_facility_using_region.columns[i + 1] ,
        ax = ax[row][col] ,
        kde = True
    )

In [ ]:
by_day_facility_using_region

In [ ]:
plt.figure(figsize = (8 , 8))

age_data = by_day_facility_using_region.iloc[1 : 18 , :]

column = age_data.columns[2 : ]

for idx in range(17):
    plt.plot(
        column ,
        age_data.iloc[idx][2 : ] ,
        label = age_data.iloc[idx][0]
    )

    plt.text(
        column[-1] ,
        age_data.iloc[idx][-1] ,
        age_data.iloc[idx][0]
    )
plt.xticks(rotation = 45)

#### 2-4. KOSIS

##### alone_percentage

In [ ]:
# alone_percentage

alone_percentage.head(5)

In [ ]:
alone_percentage.info()

In [ ]:
"""
위에 있는 인구 통계 데이터를 토대로 (행정안전부 주민등록 인구통계)
지역별로 인구중 어느 정도 독거노인 분들이 존재하는지 측정

--> 위험도를 토대로 , 지역 군집 가능할듯.
"""

region_data = alone_percentage.iloc[1: , :]

fig , ax = plt.subplots(1 , 3 , figsize = (17 , 8))
plt.subplots_adjust(wspace = 0.5)

for i in range(3):

    sns.barplot(
        data = region_data ,
        x = region_data.columns[i + 1] ,
        y = '행정구역별' ,
        ax = ax[i]
    )

### 3. 데이터 구성하기 (정리) + 파생변수 제작

#### 3-1. 파생변수 만들기

In [ ]:
# 관계성 분석하기

# 1. 인구통계 + by_region_facility

# 인구 데이터
population_data.head(3)

# by_region_facility
by_region_facility.head(3)

# 인구대비 시설 수 분석

population_decem_data = population_data[['행정구역' , '2022년12월_전체']]

population_decem_data.drop([0] , axis = 0 , inplace = True)
population_decem_data = population_decem_data.reset_index()

data_1 = pd.concat([by_region_facility , population_decem_data['2022년12월_전체']] , axis = 1)
data_1

In [ ]:
# 인구 대비 시설 수 분석

# 파생변수 제작

# 파생변수1

data_1['권역응급의료센터/인구'] = data_1['권역응급의료센터'] / data_1['2022년12월_전체']
data_1['지역응급의료센터/인구'] = data_1['지역응급의료센터'] / data_1['2022년12월_전체']
data_1['지역응급의료기관/인구'] = data_1['지역응급의료기관'] / data_1['2022년12월_전체']
data_1['응급의료시설/인구'] = data_1['응급의료시설'] / data_1['2022년12월_전체']

data_1

In [ ]:
# 시각화

fig , ax = plt.subplots(2 , 2 , figsize = (13 , 8))

for i in range(4):
    row = i // 2
    col = i % 2

    sns.barplot(
        data = data_1 ,
        x = "분류" ,
        y = data_1.columns[i + 7] ,
        ax = ax[row][col]
    )

In [ ]:
# 파생변수2

# 65세이상 인구 대비 시설수

population_decem_data = population_data[['행정구역' , '2022년12월_65세이상전체']]

population_decem_data.drop([0] , axis = 0 , inplace = True)
population_decem_data = population_decem_data.reset_index()

data_2 = pd.concat([by_region_facility , population_decem_data['2022년12월_65세이상전체']] , axis = 1)
data_2

In [ ]:
# 파생변수 생성

data_2['권역응급의료센터/65세이상인구'] = data_2['권역응급의료센터'] / data_2['2022년12월_65세이상전체']
data_2['지역응급의료센터/65세이상인구'] = data_2['지역응급의료센터'] / data_2['2022년12월_65세이상전체']
data_2['지역응급의료기관/65세이상인구'] = data_2['지역응급의료기관'] / data_2['2022년12월_65세이상전체']
data_2['응급의료시설/65세이상인구'] = data_2['응급의료시설'] / data_2['2022년12월_65세이상전체']

data_2

In [ ]:
# 시각화

fig , ax = plt.subplots(2 , 2 , figsize = (13 , 8))

for i in range(4):
    row = i // 2
    col = i % 2

    sns.barplot(
        data = data_2 ,
        x = "분류" ,
        y = data_2.columns[i + 7] ,
        ax = ax[row][col]
    )

In [ ]:
# 파생변수3

names = []
for i in range(len(new_percent.index)):
    names.append(new_percent.index[i].split()[0])

new_percent.index = names
new_percent

In [ ]:
# 파생변수 4

facility_population = by_region_facility[['분류' , '권역응급의료센터' , '지역응급의료센터' , '지역응급의료기관' , '응급의료시설']]
facility_population['권역응급의료센터/독거노인'] = facility_population['권역응급의료센터'] / alone_percentage['65세이상 1인가구(A) (가구)']
facility_population['지역응급의료센터/독거노인'] = facility_population['지역응급의료센터'] / alone_percentage['65세이상 1인가구(A) (가구)']
facility_population['지역응급의료기관/독거노인'] = facility_population['지역응급의료기관'] / alone_percentage['65세이상 1인가구(A) (가구)']
facility_population['응급의료시설/독거노인'] = facility_population['응급의료시설'] / alone_percentage['65세이상 1인가구(A) (가구)']

In [ ]:
# 시각화

fig , ax = plt.subplots(2 , 2 , figsize = (13 , 8))

for i in range(4):
    row = i // 2
    col = i % 2

    sns.barplot(
        data = facility_population ,
        x = "분류" ,
        y = facility_population.columns[i + 5] ,
        ax = ax[row][col]
    )

In [ ]:
# 파생변수 5

alone_all = alone_percentage[['행정구역별' , '65세이상 1인가구(A) (가구)']]
alone_all['65세이상 1인가구/65세이상전체'] = alone_all['65세이상 1인가구(A) (가구)'] / population_data['2022년12월_65세이상전체']
alone_all

In [ ]:
# 시각화

sns.barplot(
    data = alone_all ,
    x = '행정구역별' ,
    y = '65세이상 1인가구/65세이상전체'
)

plt.xticks(rotation = 45)

#### 3-2. DF 구성하기

In [ ]:
# 감염병 빅데이터 거래소

by_region_patient
by_region_patient_percentage

# 행정안전부 주민등록 인구통계

population_data

# 응급의료통계포털 MEDIS

by_region_facility
by_day_facility_using_region

# KOSIS

alone_percentage

# 파생변수

data_1
data_2
new_percent
facility_population
alone_all

##### 감염병 빅데이터 거래소

In [ ]:
# 데이터 구성하기

# 감염병 빅데이터 거래소

# 지역으로 변환

by_region_patient['rgn_cd'].unique()

map = {
    1 : '서울' ,
    11 : '경기' ,
    21 : '인천' ,
    31 : '충남' ,
    41 : '대구' ,
    46 : '부산' ,
    50 : '경남' ,
    54 : '전북' ,
    57 : '전남' ,
    61 : '광주' ,
    34 : '대전'
}

by_region_patient['rgn_cd'] = by_region_patient['rgn_cd'].map(map)

In [ ]:
by_region_patient_percentage['rgn_cd'] = by_region_patient_percentage['rgn_cd'].map(map)

##### 행정안전부 주민등록 인구통계

In [ ]:
# 행정안전부 주민등록 인구통계

population_data.drop([0] , axis = 0 , inplace = True)
population_data = population_data.reset_index()
population_data['행정구역'] = by_region_facility['분류']

In [ ]:
# 인구통계 데이터 대표값 설정하기

population_data.drop(['index'] , axis = 1 , inplace = True)

In [ ]:
population_data.columns[1::2]

population_data

population_data.groupby(['행정구역'])[population_data.columns[1::2]].sum()

In [ ]:
data = pd.DataFrame(population_data.iloc[: , 1::2].sum(axis = 1)).rename(columns = {
    0 : '전체 평균'
})
data = data // 12
data.index = population_data['행정구역']
data

In [ ]:
data_3 = pd.DataFrame(population_data.iloc[: , 2::2].sum(axis = 1)).rename(columns = {
    0 : '65세이상 전체평균'
})
data_3 = data_3 // 12
data_3.index = population_data['행정구역']
data_3

In [ ]:
pp_population_data = pd.concat([data , data_3] , axis = 1)
pp_population_data

##### 응급의료통계포털 MEDIS

In [ ]:
# by_region_facility

by_region_facility.drop(['권역외상센터'] , axis = 1 , inplace = True)
by_region_facility.head(3)

In [ ]:
# by_day_facility_using_region

by_day_facility_using_region.drop([0] , axis = 0 , inplace = True)
by_day_facility_using_region.drop([18] , axis = 0 , inplace = True)
by_day_facility_using_region = by_day_facility_using_region.reset_index()
by_day_facility_using_region.drop(['index'] , axis = 1 , inplace = True)
by_day_facility_using_region.head(3)

##### KOSIS

In [ ]:
alone_percentage.drop([0] , axis = 0 , inplace = True)
alone_percentage = alone_percentage.reset_index()
alone_percentage.drop(['index'] , axis = 1 , inplace = True)
alone_percentage.head(3)

##### 파생변수

In [ ]:
#data_1 **
#data_2 **
#new_percent
#facility_population **
#alone_all

In [ ]:
# new_percent

new_percent.drop(['전국'] , axis = 0 , inplace = True)
new_percent.columns = [f"{i}월 65세 이상 비율" for i in range(1 , 13)]

In [ ]:
# alone_all

alone_all.drop([0] , axis = 0 , inplace = True)
alone_all = alone_all.reset_index()
alone_all.drop(['index'] , axis = 1 , inplace = True)
alone_all.head(3)

In [ ]:
# 필요없는 데이터들 drop

data_1.drop(['권역응급의료센터' , '지역응급의료센터' , '지역응급의료기관' , '응급의료시설' ,
             '권역외상센터' , '2022년12월_전체'] , axis = 1 , inplace = True)

data_2.drop(['권역응급의료센터' , '지역응급의료센터' , '지역응급의료기관' , '응급의료시설' ,
             '권역외상센터' , '2022년12월_65세이상전체'] , axis = 1 , inplace = True)

facility_population.drop(['권역응급의료센터' , '지역응급의료센터' , '지역응급의료기관' , '응급의료시설'] , axis = 1 ,
                         inplace = True)

alone_all.drop(['65세이상 1인가구(A) (가구)'] , axis = 1 , inplace = True)

#### 최종 데이터프레임 구성하기

In [ ]:
def indexing(df , col):
    df.index = df[col]
    df.drop([col] , axis = 1 , inplace = True)
    return df

In [ ]:
by_region_facility = indexing(by_region_facility , '분류')
by_day_facility_using_region = indexing(by_day_facility_using_region , '분류')
alone_percentage = indexing(alone_percentage , '행정구역별')
data_1 = indexing(data_1 , '분류')
data_2 = indexing(data_2 , '분류')
facility_population = indexing(facility_population , '분류')
alone_all = indexing(alone_all , '행정구역별')

In [ ]:
alone_percentage.index = pp_population_data.index
new_percent.index = pp_population_data.index
alone_all.index = pp_population_data.index

In [ ]:
# 최종 데이터프레임 구성하기

df = pd.concat([pp_population_data , by_region_facility , by_day_facility_using_region ,
                alone_percentage , data_1 , data_2 , new_percent , facility_population , alone_all] ,
               axis = 1)

In [ ]:
infcd_1 = pd.DataFrame(by_region_patient.groupby(['rgn_cd'])['infcd_ptnt_cnt'].mean()).rename_axis(index = {'rgn_cd' : '분류'})
df = pd.merge(df , infcd_1 , left_index = True , right_index = True , how = 'left')
df.head(4)

In [ ]:
infcd_2 = pd.DataFrame(by_region_patient_percentage.groupby(['rgn_cd'])['infcd_ptnt_rate'].mean()).rename_axis(index = {'rgn_cd' : '분류'})
df = pd.merge(df , infcd_2 , left_index = True , right_index = True , how = 'left')
df.head(4)

In [ ]:
# NA 보기

for index in df.index:
    print(f"{index} NA : {df.loc[index].isna().sum() > 0}")

In [ ]:
# 인구수 규모를 토대로 NA값 처리

df['infcd_ptnt_cnt'] = df['infcd_ptnt_cnt'].fillna(df['infcd_ptnt_cnt'].median())
df['infcd_ptnt_rate'] = df['infcd_ptnt_rate'].fillna(df['infcd_ptnt_rate'].median())

In [ ]:
# 저장

df.to_csv('/content/drive/MyDrive/ETC/감염병 대응 빅데이터 아이디어 공모전/dataset/df.csv' , index = True)